In [9]:
import os
import glob
import cv2
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from collections import defaultdict
import re

import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [10]:
# Path to the folder containing the images you want to analyze
IMAGE_FOLDER_PATH = "Data/images/"

IMAGE_SPEC = "ARCI URAS26F1"

SUB_DIR = "/tiles"

IMAGE_PATH = os.path.join(IMAGE_FOLDER_PATH, IMAGE_SPEC + SUB_DIR)
# Path to the trained model file (.pth) saved from the training notebook
MODEL_FOLDER_PATH = "Data/segmentation_models/"

MODEL_SPEC = "obsidian_a066b033beta20.pth"

MODEL_PATH = os.path.join(MODEL_FOLDER_PATH, MODEL_SPEC)

# Path to the directory where results  will be saved
OUTPUT_PATH = "Data/inference_results"

# ---  MODEL PARAMETERS (These MUST match the model you are loading) ---
MODEL_ARC = "Unet"
ENCODER = "resnet34"
ENCODER_WEIGHTS = "imagenet"

# --- 3. PREPROCESSING PARAMETERS (These MUST match how the model was trained) ---
# Image dimensions the model was trained on
IMG_HEIGHT = 1024
IMG_WIDTH = 1024
# Number of channels in the *source* images (1 for grayscale, 3 for color).
INPUT_CHANNELS_CONFIG = 3

# --- 4. INFERENCE PARAMETERS ---
# Probability threshold to convert model output to a binary mask
PREDICTION_THRESHOLD = 0.66
# Set to None if unknown.
PIXEL_RESOLUTION_UM_PER_PX = 0.345

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
os.makedirs(OUTPUT_PATH, exist_ok=True)

if not os.path.isdir(IMAGE_PATH):
    raise FileNotFoundError(f"ERROR: Image folder not found at '{IMAGE_PATH}'")
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"ERROR: Model file not found at '{MODEL_PATH}'")

print(f"Configuration Loaded:")
print(f"  Image Folder: {IMAGE_PATH}")
print(f"  Model Path: {MODEL_PATH}")
print(f"  Output Directory: {OUTPUT_PATH}")
print(f"  Using Device: {DEVICE}")

Configuration Loaded:
  Image Folder: Data/images/ARCI URAS26F1/tiles
  Model Path: Data/segmentation_models/obsidian_a066b033beta20.pth
  Output Directory: Data/inference_results
  Using Device: cpu


In [11]:
def get_preprocessing(preprocessing_fn):
    """Combines model-specific normalization with tensor conversion."""
    _transform = [
        A.Lambda(image=preprocessing_fn),
        ToTensorV2(),
    ]
    return A.Compose(_transform)

In [12]:
print("Loading trained model...")

try:
    # Instantiate the model with the same architecture as during training
    inference_model = smp.create_model(
        arch=MODEL_ARC,
        encoder_name=ENCODER,
        encoder_weights=None,
        in_channels=INPUT_CHANNELS_CONFIG,
        classes=1,
        activation=None, 
    )
    inference_model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
    inference_model.to(DEVICE)
    inference_model.eval()
    print("Model loaded successfully and set to evaluation mode.")
except Exception as e:
    print(f"Error creating or loading model: {e}")
    print("Please ensure MODEL_ARC and ENCODER parameters in Cell 2 are correct.")
    raise e

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)
print("Preprocessing function loaded.")

Loading trained model...
Model loaded successfully and set to evaluation mode.
Preprocessing function loaded.


In [ ]:
image_files = sorted(glob.glob(os.path.join(IMAGE_PATH, '*.png')))

file_pattern = re.compile(r'Frame(\d+)_Acquisition_(\d+)_(\d+)_(\d+).png')

image_groups = defaultdict(list)

for fpath in image_files:
    fname = os.path.basename(fpath)
    match = file_pattern.match(fname)
    if match:
        n, image_id, top, left = match.groups()
        group_key = f"Frame{n}_{top}_{left}"
        image_groups[group_key].append(fpath)

print(f"Found {len(image_files)} total images.")
print(f"Grouped into {len(image_groups)} unique locations (Z-stacks). Each should have {len(list(image_groups.values())[0])} images.")

Found 2112 total images.
Grouped into 192 unique locations (Z-stacks). Each should have 11 images.


In [ ]:
print("Starting ellipse fitting...")

ellipse_analysis_results = []

for group_key, file_list in tqdm(image_groups.items(), desc="Ellipse Analysis"):
    
    group_masks = []
    
    for image_path in file_list:
        # Load and preprocess image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        sample = get_preprocessing(image=image)
        preprocessed_image = sample['image'].unsqueeze(0).to(DEVICE, dtype=torch.float32)
        
        with torch.no_grad():
            pred_output = inference_model(preprocessed_image)
            pred_probs = torch.sigmoid(pred_output)
            pr_mask = (pred_probs.squeeze().cpu().numpy().round())
        
        group_masks.append(pr_mask)
    
    # Aggregate the masks (Logical OR across the Z-stack)
    aggregated_mask = np.max(np.stack(group_masks, axis=0), axis=0).astype(np.uint8)
    
    # --- Ellipse Analysis ---
    
    # Use cv2.findContours to get the boundary of each track/blob.
    # RETR_EXTERNAL: Retrieves only the extreme outer contours.
    # CHAIN_APPROX_SIMPLE: Compresses horizontal, vertical, and diagonal segments.
    contours, _ = cv2.findContours(aggregated_mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    track_id_counter = 1
    
    for contour in contours:
        # Ellipse fitting requires a minimum of 5 points.
        if len(contour) < 5:
            continue
            
        # 1. Fit the ellipse to the contour
        # The result is: ((center_x, center_y), (minor_axis_len, major_axis_len), orientation_deg)
        ellipse = cv2.fitEllipse(contour)
        ((center_x, center_y), (axis_a_px, axis_b_px), orientation_deg) = ellipse

        # OpenCV returns the two axes in an arbitrary order, so we ensure 
        # major is the larger value and minor is the smaller value.
        major_axis_len_px = max(axis_a_px, axis_b_px)
        minor_axis_len_px = min(axis_a_px, axis_b_px)
            
        # 2. Get the area and centroid from the contour for consistency with new data
        area_px_contour = cv2.contourArea(contour)

        ellipticity = major_axis_len_px / minor_axis_len_px

        projector = np.quantile(np.nan_to_num(ellipticity, nan=0), 0.95)

        proj_len_px = major_axis_len_px * projector/ellipticity
        
        # 3. Store properties
        track_props = {
            "image_filename": os.path.basename(file_list[0]), 
            "track_id": track_id_counter,
            "area_px_contour": round(area_px_contour, 1),
            # Ellipse properties (in pixels)
            "centroid_x_px_ellipse": round(center_x, 1), 
            "centroid_y_px_ellipse": round(center_y, 1),
            "major_axis_px": round(major_axis_len_px, 2),
            "minor_axis_px": round(minor_axis_len_px, 2),
            "orientation_deg": round(orientation_deg, 2),
            "ellipticity": round(ellipticity, 2),
            "proj_len_px": round(proj_len_px, 2),

        }
        
        # Convert to physical units if resolution is known (from Cell 2)
        if PIXEL_RESOLUTION_UM_PER_PX is not None:
            track_props['proj_len_um'] = round(proj_len_px * PIXEL_RESOLUTION_UM_PER_PX, 3)

        ellipse_analysis_results.append(track_props)
        track_id_counter += 1


# 4. Convert results to a DataFrame and display a sample
df_ellipse_results = pd.DataFrame(ellipse_analysis_results)

print(f"\n--- Ellipse Fitting Results ---\nTotal tracks found across all images: {len(df_ellipse_results)}")
print("Sample of detected track and ellipse properties:")

# Display the dataframe with the new columns
df_ellipse_results.head()

Starting detailed track analysis with ellipse fitting...


Ellipse Analysis:   0%|          | 0/192 [00:00<?, ?it/s]


--- Ellipse Fitting Results ---
Total tracks found across all images: 11318
Sample of detected track and ellipse properties:


,image_filename,track_id,area_px_contour,centroid_x_px_ellipse,centroid_y_px_ellipse,major_axis_px,minor_axis_px,orientation_deg,area_um2_contour,major_axis_um,minor_axis_um
0,Frame10_Acquisition_4401_0_0.png,1,35.5,424.0,1019.6,12.45,5.70,87.17,4.23,4.294,1.966
1,Frame10_Acquisition_4401_0_0.png,2,67.5,527.8,1020.1,12.29,10.13,95.81,8.03,4.239,3.493
2,Frame10_Acquisition_4401_0_0.png,3,100.5,654.3,1019.5,15.30,10.90,97.90,11.96,5.279,3.759
3,Frame10_Acquisition_4401_0_0.png,4,16.5,561.8,1006.3,6.14,4.55,93.63,1.96,2.120,1.570
4,Frame10_Acquisition_4401_0_0.png,5,37.0,868.5,991.0,8.05,6.70,61.17,4.40,2.777,2.312
